# __Netflix Movie Ratings on Rotten Tomatoes__

## Summary:
1. Import Modules and Data
2. Data Exploration
3. Data Cleaning
    - Work around the nulls 
    - Drop irrelevant columns and/or rows
    - Rename columns
    - Create a new column to hold unique index to join on
4. Merge two dataframes to get Netflix movie ratings on Rotten Tomatoes 
    - Merge the two dataframes
    - Data Cleaning
5. Load the master file to Postgres SQL

In [1]:
import pandas as pd
import datetime
from sqlalchemy import create_engine

## Step 1-3 for Netflix Movies Dataset

### Step 1-2. Import data and data exploration

In [2]:
# import netflix data
netflix = pd.read_csv("data/netflix_titles.csv")
netflix.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China",9-Sep-19,2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,80117401,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,9-Sep-16,2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,70234439,TV Show,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,8-Sep-18,2013,TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autob..."
3,80058654,TV Show,Transformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,8-Sep-18,2016,TV-Y7,1 Season,Kids' TV,When a prison ship crash unleashes hundreds of...
4,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,8-Sep-17,2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...


### Step 3. Data Cleaning

#### Work around the nulls

In [3]:
# replace null values with "NA"

netflix_clean = netflix.fillna("NA")
netflix_clean.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China",9-Sep-19,2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,80117401,Movie,Jandino: Whatever it Takes,NA,Jandino Asporaat,United Kingdom,9-Sep-16,2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,70234439,TV Show,Transformers Prime,NA,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,8-Sep-18,2013,TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autob..."
3,80058654,TV Show,Transformers: Robots in Disguise,NA,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,8-Sep-18,2016,TV-Y7,1 Season,Kids' TV,When a prison ship crash unleashes hundreds of...
4,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,8-Sep-17,2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...


#### Drop irrelevant rows

In [4]:
# filter out tv shows based on type column as we want to focus on movie ratings

netflix_clean = netflix_clean[netflix_clean["type"] != "TV Show"]

#### Rename the columns

In [5]:
# rename column header 'listed_in' to be 'genre' and 'title' to be 'movie_title'

netflix_clean = netflix_clean.rename(columns = {"listed_in":"genre", "title":"movie_title"})

#### Create a new column to hold unique index to join on

In [6]:
# create column for unique ID (title & release year) in netflix dataframe

netflix_clean["title_release_year"]=0
netflix_clean.head()

,show_id,type,movie_title,director,cast,country,date_added,release_year,rating,duration,genre,description,title_release_year
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China",9-Sep-19,2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...,0
1,80117401,Movie,Jandino: Whatever it Takes,NA,Jandino Asporaat,United Kingdom,9-Sep-16,2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...,0
4,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,8-Sep-17,2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...,0
6,70304989,Movie,Automata,Gabe Ibáñez,"Antonio Banderas, Dylan McDermott, Melanie Gri...","Bulgaria, United States, Spain, Canada",8-Sep-17,2014,R,110 min,"International Movies, Sci-Fi & Fantasy, Thrillers","In a dystopian future, an insurance adjuster f...",0
7,80164077,Movie,Fabrizio Copano: Solo pienso en mi,"Rodrigo Toro, Francisco Schultz",Fabrizio Copano,Chile,8-Sep-17,2017,TV-MA,60 min,Stand-Up Comedy,Fabrizio Copano takes audience participation t...,0


In [7]:
# fill in title_release_year column with cleaner title_release_year value & preview

netflix_clean["title_release_year"] = netflix_clean["movie_title"]+ "_" + netflix_clean["release_year"].astype(str)
netflix_clean.head()

,show_id,type,movie_title,director,cast,country,date_added,release_year,rating,duration,genre,description,title_release_year
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China",9-Sep-19,2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...,Norm of the North: King Sized Adventure_2019
1,80117401,Movie,Jandino: Whatever it Takes,NA,Jandino Asporaat,United Kingdom,9-Sep-16,2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...,Jandino: Whatever it Takes_2016
4,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,8-Sep-17,2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...,#realityhigh_2017
6,70304989,Movie,Automata,Gabe Ibáñez,"Antonio Banderas, Dylan McDermott, Melanie Gri...","Bulgaria, United States, Spain, Canada",8-Sep-17,2014,R,110 min,"International Movies, Sci-Fi & Fantasy, Thrillers","In a dystopian future, an insurance adjuster f...",Automata_2014
7,80164077,Movie,Fabrizio Copano: Solo pienso en mi,"Rodrigo Toro, Francisco Schultz",Fabrizio Copano,Chile,8-Sep-17,2017,TV-MA,60 min,Stand-Up Comedy,Fabrizio Copano takes audience participation t...,Fabrizio Copano: Solo pienso en mi_2017


## Step 1-3 for Rotten Tomatoes Dataset

### Step 1-2. Import data and data exploration

In [8]:
# read rotten tomatoes data

rotten_tomatoes = pd.read_csv("data/rotten_tomatoes_movies.csv")
rotten_tomatoes.head()

,rotten_tomatoes_link,movie_title,movie_info,critics_consensus,poster_image_url,rating,genre,directors,writers,cast,...,studio_name,tomatometer_status,tomatometer_rating,tomatometer_count,audience_status,audience_rating,audience_count,audience_top_critics_count,audience_fresh_critics_count,audience_rotten_critics_count
0,/m/0814255,Percy Jackson & the Olympians: The Lightning T...,A teenager discovers he's the descendant of a ...,Though it may seem like just another Harry Pot...,https://resizing.flixster.com/p1veUpQ4ktsSHtRu...,PG,"Action & Adventure, Comedy, Drama, Science Fic...",Chris Columbus,Craig Titley,"Logan Lerman, Brandon T. Jackson, Alexandra Da...",...,20th Century Fox,Rotten,49,144,Spilled,53.0,254287.0,38,71,73
1,/m/0878835,Please Give,Kate has a lot on her mind. There's the ethics...,Nicole Holofcener's newest might seem slight i...,https://resizing.flixster.com/0AbudQ4KsB4BeXSB...,R,Comedy,Nicole Holofcener,Nicole Holofcener,"Catherine Keener, Amanda Peet, Oliver Platt, R...",...,Sony Pictures Classics,Certified Fresh,86,140,Upright,64.0,11567.0,43,121,19
2,/m/10,10,Blake Edwards' 10 stars Dudley Moore as George...,NaN,https://resizing.flixster.com/mF0dxH6UTa0FdkMs...,R,"Comedy, Romance",Blake Edwards,Blake Edwards,"Dudley Moore, Bo Derek, Julie Andrews, Robert ...",...,Waner Bros.,Fresh,68,22,Spilled,53.0,14670.0,2,15,7
3,/m/1000013-12_angry_men,12 Angry Men (Twelve Angry Men),"A Puerto Rican youth is on trial for murder, a...",Sidney Lumet's feature debut is a superbly wri...,https://resizing.flixster.com/u-8xAyGaDVvROLiR...,NR,"Classics, Drama",Sidney Lumet,Reginald Rose,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",...,Criterion Collection,Certified Fresh,100,51,Upright,97.0,105000.0,6,51,0
4,/m/1000079-20000_leagues_under_the_sea,"20,000 Leagues Under The Sea","This 1954 Disney version of Jules Verne's 20,0...","One of Disney's finest live-action adventures,...",https://resizing.flixster.com/FKExgYBHu07XLoil...,G,"Action & Adventure, Drama, Kids & Family",Richard Fleischer,Earl Felton,"James Mason, Kirk Douglas, Paul Lukas, Peter L...",...,Disney,Fresh,89,27,Upright,74.0,68860.0,5,24,3


### Step 3. Data Cleaning

#### work around the nulls

In [9]:
# drop null values
rotten_tomatoes.dropna(inplace=True)

rotten_tomatoes.head()

,rotten_tomatoes_link,movie_title,movie_info,critics_consensus,poster_image_url,rating,genre,directors,writers,cast,...,studio_name,tomatometer_status,tomatometer_rating,tomatometer_count,audience_status,audience_rating,audience_count,audience_top_critics_count,audience_fresh_critics_count,audience_rotten_critics_count
0,/m/0814255,Percy Jackson & the Olympians: The Lightning T...,A teenager discovers he's the descendant of a ...,Though it may seem like just another Harry Pot...,https://resizing.flixster.com/p1veUpQ4ktsSHtRu...,PG,"Action & Adventure, Comedy, Drama, Science Fic...",Chris Columbus,Craig Titley,"Logan Lerman, Brandon T. Jackson, Alexandra Da...",...,20th Century Fox,Rotten,49,144,Spilled,53.0,254287.0,38,71,73
1,/m/0878835,Please Give,Kate has a lot on her mind. There's the ethics...,Nicole Holofcener's newest might seem slight i...,https://resizing.flixster.com/0AbudQ4KsB4BeXSB...,R,Comedy,Nicole Holofcener,Nicole Holofcener,"Catherine Keener, Amanda Peet, Oliver Platt, R...",...,Sony Pictures Classics,Certified Fresh,86,140,Upright,64.0,11567.0,43,121,19
3,/m/1000013-12_angry_men,12 Angry Men (Twelve Angry Men),"A Puerto Rican youth is on trial for murder, a...",Sidney Lumet's feature debut is a superbly wri...,https://resizing.flixster.com/u-8xAyGaDVvROLiR...,NR,"Classics, Drama",Sidney Lumet,Reginald Rose,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",...,Criterion Collection,Certified Fresh,100,51,Upright,97.0,105000.0,6,51,0
4,/m/1000079-20000_leagues_under_the_sea,"20,000 Leagues Under The Sea","This 1954 Disney version of Jules Verne's 20,0...","One of Disney's finest live-action adventures,...",https://resizing.flixster.com/FKExgYBHu07XLoil...,G,"Action & Adventure, Drama, Kids & Family",Richard Fleischer,Earl Felton,"James Mason, Kirk Douglas, Paul Lukas, Peter L...",...,Disney,Fresh,89,27,Upright,74.0,68860.0,5,24,3
5,/m/10000_bc,"10,000 B.C.",A young outcast from a primitive tribe is forc...,With attention strictly paid to style instead ...,https://resizing.flixster.com/x41Imwhe6xccM0Zl...,PG-13,"Action & Adventure, Classics, Drama",Roland Emmerich,"Roland Emmerich, Harald Kloser","Steven Strait, Camilla Belle, Cliff Curtis, Jo...",...,Warner Bros. Pictures,Rotten,8,148,Spilled,37.0,411087.0,36,12,136


In [10]:
# fill null columns as "NA"

RT_clean = rotten_tomatoes.fillna("NA")
RT_clean.head()

,rotten_tomatoes_link,movie_title,movie_info,critics_consensus,poster_image_url,rating,genre,directors,writers,cast,...,studio_name,tomatometer_status,tomatometer_rating,tomatometer_count,audience_status,audience_rating,audience_count,audience_top_critics_count,audience_fresh_critics_count,audience_rotten_critics_count
0,/m/0814255,Percy Jackson & the Olympians: The Lightning T...,A teenager discovers he's the descendant of a ...,Though it may seem like just another Harry Pot...,https://resizing.flixster.com/p1veUpQ4ktsSHtRu...,PG,"Action & Adventure, Comedy, Drama, Science Fic...",Chris Columbus,Craig Titley,"Logan Lerman, Brandon T. Jackson, Alexandra Da...",...,20th Century Fox,Rotten,49,144,Spilled,53.0,254287.0,38,71,73
1,/m/0878835,Please Give,Kate has a lot on her mind. There's the ethics...,Nicole Holofcener's newest might seem slight i...,https://resizing.flixster.com/0AbudQ4KsB4BeXSB...,R,Comedy,Nicole Holofcener,Nicole Holofcener,"Catherine Keener, Amanda Peet, Oliver Platt, R...",...,Sony Pictures Classics,Certified Fresh,86,140,Upright,64.0,11567.0,43,121,19
3,/m/1000013-12_angry_men,12 Angry Men (Twelve Angry Men),"A Puerto Rican youth is on trial for murder, a...",Sidney Lumet's feature debut is a superbly wri...,https://resizing.flixster.com/u-8xAyGaDVvROLiR...,NR,"Classics, Drama",Sidney Lumet,Reginald Rose,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",...,Criterion Collection,Certified Fresh,100,51,Upright,97.0,105000.0,6,51,0
4,/m/1000079-20000_leagues_under_the_sea,"20,000 Leagues Under The Sea","This 1954 Disney version of Jules Verne's 20,0...","One of Disney's finest live-action adventures,...",https://resizing.flixster.com/FKExgYBHu07XLoil...,G,"Action & Adventure, Drama, Kids & Family",Richard Fleischer,Earl Felton,"James Mason, Kirk Douglas, Paul Lukas, Peter L...",...,Disney,Fresh,89,27,Upright,74.0,68860.0,5,24,3
5,/m/10000_bc,"10,000 B.C.",A young outcast from a primitive tribe is forc...,With attention strictly paid to style instead ...,https://resizing.flixster.com/x41Imwhe6xccM0Zl...,PG-13,"Action & Adventure, Classics, Drama",Roland Emmerich,"Roland Emmerich, Harald Kloser","Steven Strait, Camilla Belle, Cliff Curtis, Jo...",...,Warner Bros. Pictures,Rotten,8,148,Spilled,37.0,411087.0,36,12,136


#### Rename the columns

In [11]:
# rename 'in_theaters_date' to 'RT_release_year'

RT_clean = RT_clean.rename(columns = {"in_theaters_date":"RT_release_date"})
RT_clean.head()

,rotten_tomatoes_link,movie_title,movie_info,critics_consensus,poster_image_url,rating,genre,directors,writers,cast,...,studio_name,tomatometer_status,tomatometer_rating,tomatometer_count,audience_status,audience_rating,audience_count,audience_top_critics_count,audience_fresh_critics_count,audience_rotten_critics_count
0,/m/0814255,Percy Jackson & the Olympians: The Lightning T...,A teenager discovers he's the descendant of a ...,Though it may seem like just another Harry Pot...,https://resizing.flixster.com/p1veUpQ4ktsSHtRu...,PG,"Action & Adventure, Comedy, Drama, Science Fic...",Chris Columbus,Craig Titley,"Logan Lerman, Brandon T. Jackson, Alexandra Da...",...,20th Century Fox,Rotten,49,144,Spilled,53.0,254287.0,38,71,73
1,/m/0878835,Please Give,Kate has a lot on her mind. There's the ethics...,Nicole Holofcener's newest might seem slight i...,https://resizing.flixster.com/0AbudQ4KsB4BeXSB...,R,Comedy,Nicole Holofcener,Nicole Holofcener,"Catherine Keener, Amanda Peet, Oliver Platt, R...",...,Sony Pictures Classics,Certified Fresh,86,140,Upright,64.0,11567.0,43,121,19
3,/m/1000013-12_angry_men,12 Angry Men (Twelve Angry Men),"A Puerto Rican youth is on trial for murder, a...",Sidney Lumet's feature debut is a superbly wri...,https://resizing.flixster.com/u-8xAyGaDVvROLiR...,NR,"Classics, Drama",Sidney Lumet,Reginald Rose,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",...,Criterion Collection,Certified Fresh,100,51,Upright,97.0,105000.0,6,51,0
4,/m/1000079-20000_leagues_under_the_sea,"20,000 Leagues Under The Sea","This 1954 Disney version of Jules Verne's 20,0...","One of Disney's finest live-action adventures,...",https://resizing.flixster.com/FKExgYBHu07XLoil...,G,"Action & Adventure, Drama, Kids & Family",Richard Fleischer,Earl Felton,"James Mason, Kirk Douglas, Paul Lukas, Peter L...",...,Disney,Fresh,89,27,Upright,74.0,68860.0,5,24,3
5,/m/10000_bc,"10,000 B.C.",A young outcast from a primitive tribe is forc...,With attention strictly paid to style instead ...,https://resizing.flixster.com/x41Imwhe6xccM0Zl...,PG-13,"Action & Adventure, Classics, Drama",Roland Emmerich,"Roland Emmerich, Harald Kloser","Steven Strait, Camilla Belle, Cliff Curtis, Jo...",...,Warner Bros. Pictures,Rotten,8,148,Spilled,37.0,411087.0,36,12,136


#### Drop irrelevant columns

In [12]:
# keep only certain columns in RT_clean dataframe

RT_clean = RT_clean[["rotten_tomatoes_link", "movie_title", "RT_release_date", "tomatometer_status", "tomatometer_rating", "audience_rating"]]
RT_clean.head()

,rotten_tomatoes_link,movie_title,RT_release_date,tomatometer_status,tomatometer_rating,audience_rating
0,/m/0814255,Percy Jackson & the Olympians: The Lightning T...,2/12/2010,Rotten,49,53.0
1,/m/0878835,Please Give,4/30/2010,Certified Fresh,86,64.0
3,/m/1000013-12_angry_men,12 Angry Men (Twelve Angry Men),4/13/1957,Certified Fresh,100,97.0
4,/m/1000079-20000_leagues_under_the_sea,"20,000 Leagues Under The Sea",1/1/1954,Fresh,89,74.0
5,/m/10000_bc,"10,000 B.C.",3/7/2008,Rotten,8,37.0


#### Create a new column to hold unique index to join on

In [13]:
# concatenate movie title and release year into title_release_year column as key value

RT_clean_1 = RT_clean.copy()
RT_clean_1["title_release_year"] = RT_clean["movie_title"] + "_" + RT_clean['RT_release_date'].str[:4]
RT_clean_1.head()

,rotten_tomatoes_link,movie_title,RT_release_date,tomatometer_status,tomatometer_rating,audience_rating,title_release_year
0,/m/0814255,Percy Jackson & the Olympians: The Lightning T...,2/12/2010,Rotten,49,53.0,Percy Jackson & the Olympians: The Lightning T...
1,/m/0878835,Please Give,4/30/2010,Certified Fresh,86,64.0,Please Give_4/30
3,/m/1000013-12_angry_men,12 Angry Men (Twelve Angry Men),4/13/1957,Certified Fresh,100,97.0,12 Angry Men (Twelve Angry Men)_4/13
4,/m/1000079-20000_leagues_under_the_sea,"20,000 Leagues Under The Sea",1/1/1954,Fresh,89,74.0,"20,000 Leagues Under The Sea_1/1/"
5,/m/10000_bc,"10,000 B.C.",3/7/2008,Rotten,8,37.0,"10,000 B.C._3/7/"


### 4. Merge two dataframes to get Netflix movie ratings on Rotten Tomatoes 

### Merge two dataframes

In [14]:
# join two dataframes together based on movie title and year

master_df = pd.merge(netflix_clean, RT_clean_1, on = "title_release_year", how = "inner")
master_df.head()

,show_id,type,movie_title_x,director,cast,country,date_added,release_year,rating,duration,genre,description,title_release_year,rotten_tomatoes_link,movie_title_y,RT_release_date,tomatometer_status,tomatometer_rating,audience_rating


In [15]:
# how many rows are there in master_df
master_df.shape

(0, 19)

#### Data Cleaning

In [16]:
# drop 'movie_title_y' and 'RT_release_date'
master_df = master_df.drop(columns = ["movie_title_y","RT_release_date"])
master_df

,show_id,type,movie_title_x,director,cast,country,date_added,release_year,rating,duration,genre,description,title_release_year,rotten_tomatoes_link,tomatometer_status,tomatometer_rating,audience_rating


In [17]:
# rename "movie_title_x" column to be "movie_title"

master_df = master_df.rename(columns = {"movie_title_x":"movie_title"})
master_df

,show_id,type,movie_title,director,cast,country,date_added,release_year,rating,duration,genre,description,title_release_year,rotten_tomatoes_link,tomatometer_status,tomatometer_rating,audience_rating


### 5. Load the master file to Postgres

In [18]:
# Create database connection
# Make sure you create the database in Postgres first
connection_string = "postgres:postgres@localhost:5432/netflix_RT_ratings"
engine = create_engine(f'postgresql://{connection_string}')

In [19]:
# Use pandas to load json converted DataFrame into database
master_df.to_sql(name='netflix_RT_ratings', con=engine, if_exists = 'replace', index=True)

OperationalError: (psycopg2.OperationalError) FATAL:  database "netflix_RT_ratings" does not exist

(Background on this error at: http://sqlalche.me/e/e3q8)